# Joining and Appending DataFrames in PySpark HW

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [14]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

spark: SparkSession = SparkSession.builder.master("local[*]").appName("JoinHW").getOrCreate()

spark

## Read in the database

Let cotinue working with our college courses dataframe to get some more insights and practice what we have learned!Let's read in the whole database using the loop function that we learned about in the lecture to automatically read in all the datasets from the uw-madision-courses folder (there are too many datasets to each one individually.

In [15]:
import os

dfs = {}

base_path = r"Datasets/uw-madison-courses"

for f in os.listdir(base_path):
    f_name = f.split(".")[0]
    f_type = f.split(".")[-1]

    if f_type == "csv":
        dfs[f_name] = spark.read.csv(
            inferSchema=True,
            header=True,
            sep=",",
            path=f"{base_path}/{f}"
        )

Now check the contents of a few of the dataframses that were read in above.

In [ ]:
list(dfs.keys())

## Recap: About this database

You will notice that there are several more tables in the uw-madision-courses folder than there are read in above. This so that you will have a chance to practice your own custom joins and learn about the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! I'll save that for the HW :) 

Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
 **Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
So alright, let's use this information to discover some insights from this data!

## 1a. Can you assign the room numbers to each section of each course?

Show only the rooms uuid, facility code, room number, term code and the name of the course from the course_offerings table.

In [16]:
print(dfs.get("course_offerings").columns)
print(dfs.get("sections").columns)
print(dfs.get("rooms").columns)

course_offerings = dfs.get("course_offerings")
sections = dfs.get("sections")
rooms = dfs.get("rooms")

['uuid', 'course_uuid', 'term_code', 'name']
['uuid', 'course_offering_uuid', 'section_type', 'number', 'room_uuid', 'schedule_uuid']
['uuid', 'facility_code', 'room_code']


In [19]:
ex1 = (
    course_offerings
    .join(
        other=sections,
        on=course_offerings.uuid == sections.course_offering_uuid
    )
    .join(
        other=rooms,
        on=sections.room_uuid == rooms.uuid
    )
    .select(
        course_offerings.name,
        "room_uuid",
        "facility_code",
        F.col("number").alias("section_number"),
        "term_code"
    )
    .orderBy(
        "name"
    )
)

ex1.toPandas()

,name,room_uuid,facility_code,section_number,term_code
0,"""""""Scand Modern"""" Phenomenon""",3215b86b-a4e4-39fe-9818-7e6e75569076,0400,1,1112
1,17th C Literature-1600-1660,b4bfb1f2-4ecd-361b-bb4f-a7f46e62602f,0048,1,1082
2,17th C Literature-1600-1660,165c677f-2362-3405-b219-c122ef48bfb9,0048,1,1072
3,17th-Century Literature,1047a29e-32b3-3a5b-8877-5cad2be098d1,0018,1,1152
4,18th Century British Novel,c25fbc34-2af7-3774-b306-34ebf6acfecf,0018,1,1112
...,...,...,...,...,...
209639,null,bc6e5cc5-1630-37be-aa3c-9a9ad3883a64,0482,1,1084
209640,null,529e51c9-ce17-3df3-866e-836fc9694b93,0482,301,1084
209641,null,4951625e-9f56-397d-8b2f-b773bf40c97a,0056,1,1182
209642,null,86d1debf-3f3a-39b7-bc0d-976a4fa33862,ONLINE,1,1182


## 1b. Now show same output as above but for only facility number 0469 (facility_code)

In [20]:
ex1b = (
    ex1
    .select(
        "*"
    )
    .where(
        F.col("facility_code") == "0469"
    )
)

ex1b.toPandas()

,name,room_uuid,facility_code,section_number,term_code
0,18th-Century English Novel,9a15c09e-57d8-3658-ae72-aae5ffdef108,0469,1,1174
1,19th Century Philosophers,c731d9db-3bc4-3436-a5fd-7954098d3193,0469,1,1114
2,1st Yr Classical Chinese,f4ccf94a-5c59-3a6d-97e9-d44c4ca6a0fc,0469,1,1092
3,1st Yr Classical Chinese,f4ccf94a-5c59-3a6d-97e9-d44c4ca6a0fc,0469,1,1092
4,1st-Yr Seminar: Biological Sci,2b57b421-6c9f-3d88-9299-21b41c693446,0469,3,1152
...,...,...,...,...,...
16328,null,dfb7f39a-210b-380c-a162-64673d7ee85b,0469,1,1144
16329,null,dfb7f39a-210b-380c-a162-64673d7ee85b,0469,1,1172
16330,null,66dde0f0-c802-3a42-9f25-870dd24e55eb,0469,1,1174
16331,null,880a148b-1709-336f-9345-7b18f4559b48,0469,1,1174


## 2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid

In [ ]:
rooms = dfs.get("rooms")
subjects = dfs.get("subjects")
subject_memberships = dfs.get("subject_memberships")
sections = dfs.get("sections")

print(rooms.columns)
print(subjects.columns)
print(subject_memberships.columns)
print(sections.columns)

In [29]:
ex2 = (
    sections
    .join(
        other=rooms,
        on=sections.room_uuid == rooms.uuid
    )
    .join(
        other=subject_memberships,
        on=sections.course_offering_uuid == subject_memberships.course_offering_uuid
    )
    .join(
        other=subjects,
        on=subject_memberships.subject_code == subjects.code
    )
)

ex2.toPandas()

(
    ex2
    .groupBy(
        F.col("facility_code"),
        F.col("name"),
    )
    .agg(
        F.count("section_type").alias("n_sections")
    )
    .select(
        "facility_code",
        "name",
        "n_sections"
    )
    .orderBy(
        "facility_code",
        "name",
    )
    .toPandas()
)

,facility_code,name,n_sections
0,0000,ANIMAL HEALTH AND BIOMEDICAL SCIENCES,2
1,0000,African Languages and Literature,1
2,0000,Agroecology,1
3,0000,Agronomy,2
4,0000,Animal Sciences,2
...,...,...,...
3961,ONLINE,Rehabilitation Psychology and Special Education,29
3962,ONLINE,Religious Studies,2
3963,ONLINE,Social Work,59
3964,ONLINE,Sociology,4


## 3. What are the hardest classes?

Let's see if we can figure out which classes are the hardest by seeing how many students failed. Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offering table.

   - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid

In [36]:
grade_dist = dfs.get("grade_distributions")
course_offerings = dfs.get("course_offerings")

(
    grade_dist
    .groupBy(
        "course_offering_uuid"
    )
    .agg(
        F.sum("f_count").alias("failed")
    )
    .join(
        other=course_offerings,
        on=grade_dist.course_offering_uuid == course_offerings.uuid
    )
    .select(
        "name",
        F.col("failed")
    ).
    orderBy(
        "name",
        "failed"
    )
    .toPandas()
)

,name,failed
0,"""""""Scand Modern"""" Phenomenon""",0
1,17th C Literature-1600-1660,0
2,17th C Literature-1600-1660,0
3,17th-Century Literature,0
4,18th Century British Novel,0
...,...,...
80166,null,2
80167,null,2
80168,null,2
80169,null,2


,course_offering_uuid,section_number,f_count,uuid,course_offering_uuid,section_type,number,room_uuid,schedule_uuid
0,344b3ebe-da7e-314c-83ed-9425269695fd,1,0,45adf63c-48c9-3659-8561-07556d2d4ddf,344b3ebe-da7e-314c-83ed-9425269695fd,FLD,1,null,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,1,0,c6280e23-5e43-3859-893e-540d94993529,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,FLD,1,null,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
2,ea3b717c-d66b-30dc-8b37-964d9688295f,1,0,9395dc21-15d1-3fab-8d1f-6f3fe6114c48,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,04368a56-c959-3e4b-8b3d-f4cc3538fea5,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
3,ea3b717c-d66b-30dc-8b37-964d9688295f,1,0,b99e440b-39db-350a-81eb-b6eb1bd8b0bc,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,null,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
4,ea3b717c-d66b-30dc-8b37-964d9688295f,1,0,ca1c841f-41d5-3298-bd25-01c65751f150,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,null,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
...,...,...,...,...,...,...,...,...,...
2077511,3573e335-e7be-33a6-b023-8704b3acc93b,1,0,83b4bb4a-a54e-328b-98eb-3f17571d278e,3573e335-e7be-33a6-b023-8704b3acc93b,SEM,1,33f69fe2-fe56-3b35-9302-7b6d1954b556,1c706999-eedc-3cd3-8595-097051da2fa2
2077512,3573e335-e7be-33a6-b023-8704b3acc93b,1,0,62be3a59-6ff1-39cb-b5a3-433e317b952e,3573e335-e7be-33a6-b023-8704b3acc93b,SEM,1,33f69fe2-fe56-3b35-9302-7b6d1954b556,1c706999-eedc-3cd3-8595-097051da2fa2
2077513,3573e335-e7be-33a6-b023-8704b3acc93b,1,0,78135ed6-31b3-3973-99d5-fd98ab9afef3,3573e335-e7be-33a6-b023-8704b3acc93b,SEM,1,409b3445-80fc-3ca2-a44e-f254ba48d5cd,eaab6f3c-0f7b-36c0-8175-56207644338e
2077514,13278922-0726-3823-abda-dc14df03d8a6,7,0,50f9cf79-a4e3-3975-a292-b130d3d5d97c,13278922-0726-3823-abda-dc14df03d8a6,LEC,7,null,739ab54a-17e8-3428-b64a-5b60640a35ff


## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [9]:
values = [('Monkey', 10), ('Monkay', 36), ('Mnky', 123), \
          ('Elephant', 48), ('Elefant', 16), ('Ellafant', 1), \
          ('Hippopotamus', 48), ('Hipopotamus', 16), ('Hippo', 1)]
zoo = spark.createDataFrame(values, ['Animal', 'age'])
zoo.show()

+------------+---+
|      Animal|age|
+------------+---+
|      Monkey| 10|
|      Monkay| 36|
|        Mnky|123|
|    Elephant| 48|
|     Elefant| 16|
|    Ellafant|  1|
|Hippopotamus| 48|
| Hipopotamus| 16|
|       Hippo|  1|
+------------+---+



In [37]:
options = spark.createDataFrame(['Monkey', 'Elephant', 'Hippopotamus'], T.StringType())

In [39]:
results = (
    zoo
    .join(
        other=options,
        on=F.levenshtein(zoo.Animal, options.value) < 5,
    )
)

results.toPandas()

,Animal,age,value
0,Monkey,10,Monkey
1,Monkay,36,Monkey
2,Mnky,123,Monkey
3,Elephant,48,Elephant
4,Elefant,16,Elephant
5,Ellafant,1,Elephant
6,Hippopotamus,48,Hippopotamus
7,Hipopotamus,16,Hippopotamus


### Great job! 